<h1 style="font-family: 'Georgia', serif; text-align: center; font-size: 30px; color:rgb(52, 152, 219); text-shadow: 2px 2px 5px rgba(0, 0, 0, 0.3)">Préparation des données</h1>
<br/>
<br/>
  

<h1 style="font-family: 'Georgia', serif; font-size: 30px; color:rgb(52, 152, 219); text-shadow: 2px 2px 5px rgba(0, 0, 0, 0.3)">INTRODUCTION</h1>
<br/>

  

<p>
Le marché de l'immobilier est un secteur économique les plus importants, rentable et stable. Un investisseur averti utiliserait les outils Data Science  afin d'analyser les données et d'en tirer des informations utiles telles que l'évolution des prix, l'accessibilité des logements... 
</p>
<p >Dans ce projet, nous allons construire un modèle de regression linéaire afin de prédire le prix d'une maison en fonction de certains attributs
</p>

In [4]:
#librairies
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

## tout dabord je modifie le repertoire afin d'importer les données

In [16]:
import os

# Obtenir le répertoire de travail actuel
current_directory = os.getcwd()
print(f"Répertoire actuel : {current_directory}")

# Remplacer "src" par "data/HousingData.csv"
if "src" in current_directory:
    new_directory = current_directory.replace("src", "data/HousingData.csv")
    print(f"Nouveau chemin : {new_directory}")
    
    # Changer le répertoire courant
    new_path = os.path.dirname(new_directory)  # On ne peut se déplacer que dans un répertoire, pas un fichier
    os.chdir(new_path)
    print(f"Répertoire actuel après modification : {os.getcwd()}")
else:
    print("Le répertoire actuel ne contient pas 'src'.")

Répertoire actuel : /home/onyxia/work/prediction_prix_maison/src
Nouveau chemin : /home/onyxia/work/prediction_prix_maison/data/HousingData.csv
Répertoire actuel après modification : /home/onyxia/work/prediction_prix_maison/data


In [19]:
df= pd.read_csv(new_directory)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     486 non-null    float64
 1   ZN       486 non-null    float64
 2   INDUS    486 non-null    float64
 3   CHAS     486 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      486 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    int64  
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    486 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


<h1 style="font-family: 'Georgia', serif; text-align: center; font-size: 30px; color:rgb(52, 152, 219); text-shadow: 2px 2px 5px rgba(0, 0, 0, 0.3)">Analyse Descriptive des données</h1>
<br/>
<br/>
  